# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = "output_data/my_weather_data.csv"
weather_data = pd.read_csv(data)
weather_df = pd.DataFrame(weather_data)
weather_df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jiaozuo,35.2397,113.2331,80.85,78,100,7.38,CN,1627473600
1,1,Mount Isa,-20.7333,139.5000,75.79,27,12,4.70,AU,1627473600
2,2,Mar del Plata,-38.0023,-57.5575,42.58,65,80,25.72,AR,1627473600
3,3,Ostrovnoy,68.0531,39.5131,48.38,83,99,9.98,RU,1627473600
4,4,Igarka,67.4667,86.5833,55.33,79,100,9.19,RU,1627473600


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [16]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
hum_rate = weather_df["Humidity"].astype(float)

In [22]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("Images/vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Define perfect weather conditions less than 90 but greater than 75
first = weather_df.loc[weather_df["Max Temp (F)"]<90,:]
second = first.loc[weather_df["Max Temp (F)"]>75,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third.head()

#Cloudiness less than 10.
fourth = third.loc[third["Cloudiness"]<10,:]
fourth= round(fourth,2)
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
34,34,Batesville,35.77,-91.64,75.15,95,4,1.59,US,1627473600
106,106,Porto Torres,40.84,8.40,88.99,51,0,8.39,IT,1627473600
133,133,Vanavara,60.34,102.28,88.97,31,7,6.58,RU,1627473600
134,134,Sumbe,-11.21,13.84,76.64,70,6,9.60,AO,1627473600
175,175,San Patricio,28.02,-97.52,78.08,84,2,2.44,US,1627473600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame()
hotel_df["Hotel Name"]=""
hotel_df["Latitude"]=""
hotel_df["Longitude"]=""
hotel_df["Country"]=fourth["Country"]
hotel_df["City"]= fourth["City"]
hotel_df

,Hotel Name,Latitude,Longitude,Country,City
34,NaN,NaN,NaN,US,Batesville
106,NaN,NaN,NaN,IT,Porto Torres
133,NaN,NaN,NaN,RU,Vanavara
134,NaN,NaN,NaN,AO,Sumbe
175,NaN,NaN,NaN,US,San Patricio
187,NaN,NaN,NaN,BR,Urucará
189,NaN,NaN,NaN,NaN,Opuwo
231,NaN,NaN,NaN,MX,Santa Rosalía
244,NaN,NaN,NaN,PF,Haapiti
254,NaN,NaN,NaN,PT,Lata


In [8]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in fourth.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row['City']
    country = row['Country']


    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(response.url)

#     # convert to json
    hotels = response.json()
#     print(json.dumps(hotels, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][7]['name']
        hotel_df.loc[index, "Longitude"] = hotels["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Latitude"] = hotels["results"][0]["geometry"]["location"]["lat"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
hotel_df = hotel_df.dropna()
hotel_df.head()

,Hotel Name,Latitude,Longitude,Country,City
34,Brecks RV Park,35.7914,-91.6394,US,Batesville
106,Il Melo Residence,40.8387,8.40172,IT,Porto Torres
134,Complexo Hoteleiro Mar Sol,-11.1982,13.8354,AO,Sumbe
175,Sinton RV Park LLC,28.0303,-97.5297,US,San Patricio
231,REC FRIREN cctv,27.3291,-112.256,MX,Santa Rosalía


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [35]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))